In [1]:
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
import zipfile

2021-09-11 02:03:46.226400: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-11 02:03:46.226417: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
with zipfile.ZipFile('HAND.zip') as existing_zip:
    existing_zip.extractall()

In [2]:
def get_train_data(path, class_names):
    datalists = []
    datalabellists = []
    for i, classname in enumerate(class_names):
        tmplist = []
        for csvfile in glob.glob(path+str(classname)+'/*.csv'):
            tmp = np.loadtxt(csvfile, delimiter=',', dtype='float64')
            tmplist.append(tmp)
        datalist = np.concatenate(tmplist)
        datalists.append(datalist)
        datalabellist = np.full(datalist.shape[0], i)
        print(len(datalabellist))
        datalabellists.append(datalabellist)
    train_data = np.concatenate(datalists)
    train_labels = np.concatenate(datalabellists)
    return train_data, train_labels

In [3]:
def save_model(model, path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

In [4]:
class_names = ['LEFTFINGER', 'LEFTOK', 'LEFTPAPER', 'LEFTSCISSORS', 'LEFTSTONE', 'RIGHTFINGER', 'RIGHTOK', 'RIGHTPAPER', 'RIGHTSCISSORS', 'RIGHTSTONE']

In [5]:
train_data, train_labels = get_train_data('./HAND/', class_names)

748
1340
1170
1300
840
810
1090
1060
1798
1139


In [6]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(42, 1)),
    keras.layers.Dense(21, activation='relu'),
    keras.layers.Dense(len(class_names), activation='softmax')
])

In [7]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(train_data, train_labels, epochs=5)

Epoch 1/5
353/353 [==============================] - 1s 816us/step - loss: 1.8969 - accuracy: 0.3758
Epoch 2/5
353/353 [==============================] - 0s 848us/step - loss: 0.5216 - accuracy: 0.9619
Epoch 3/5
353/353 [==============================] - 0s 888us/step - loss: 0.2129 - accuracy: 0.9801
Epoch 4/5
353/353 [==============================] - 0s 881us/step - loss: 0.1239 - accuracy: 0.9843
Epoch 5/5
353/353 [==============================] - 0s 847us/step - loss: 0.0898 - accuracy: 0.9858


In [9]:
save_model(model, '/home/hands.tflite')

INFO:tensorflow:Assets written to: /tmp/tmpoisq81am/assets
